# Bike Sharing Demand

This project uses LSTM neural networks to predict bike-sharing demand. The model processes historical data including weather and temporal features through MinMaxScaler normalization, and outputs hourly bike rental predictions. The implementation achieves practical results with a simple architecture of one LSTM layer followed by dropout regularization.

Dataset: https://www.kaggle.com/competitions/bike-sharing-demand/data

Hugging Face: https://huggingface.co/spaces/alperugurcan/bike-demand-predictor

In [4]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

train_df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
sample_submission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

features = train_df.drop(columns=['count', 'datetime', 'casual', 'registered'])
target = train_df['count']
test_features = test_df.drop(columns=['datetime'])

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(features)
y_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1))
test_scaled = scaler_x.transform(test_features)

model = Sequential([
    LSTM(30, activation='relu', input_shape=(1, X_scaled.shape[1])),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])
y = y_scaled

model.fit(X_reshaped, y, epochs=50, batch_size=15, validation_split=0.2)

test_reshaped = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])
predictions_scaled = model.predict(test_reshaped)
predictions = scaler_y.inverse_transform(predictions_scaled)

submission = sample_submission.copy()
submission['count'] = np.maximum(0, predictions.flatten().astype(int))  # Ensure non-negative values
submission.to_csv('submission.csv', index=False)

print("Submission created with shape:", submission.shape)
print("Sample predictions:", submission['count'].head())

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7cdaf1b17e80>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [3]:
model.save('bike_model.h5')
joblib.dump(scaler_x, 'scaler_x.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')
np.save('feature_names.npy', features.columns.values)